# Mess3 generator smoke test

This notebook builds a simple `mess3` hidden Markov model and prints a few generated symbols.


In [1]:
# pyright: reportMissingImports=false
import jax

from simplexity.generative_processes.builder import build_hidden_markov_model

# Instantiate mess3 with defaults from config (x=0.15, a=0.6)
model = build_hidden_markov_model("mess3", x=0.15, a=0.6)

# Initial belief state: use the model's normalizing_eigenvector as a valid distribution
state = model.normalizing_eigenvector[None, :]

# Generate a short sequence
key = jax.random.PRNGKey(0)[None, :]
sequence_len = 10
final_state, observations = model.generate(state, key, sequence_len, False)

print("Generated observations:", observations.squeeze().tolist())


Generated observations: [0, 2, 0, 2, 2, 0, 0, 1, 2, 1]


In [ ]:
# pyright: reportMissingImports=false
# Generate again but keep all intermediate belief states (priors)
states_per_step, obs = model.generate(state, key, sequence_len, True)

# Normalize belief states to probabilities per step
beliefs = jax.vmap(model.normalize_belief_state)(states_per_step.squeeze(0))

for t, (o, b) in enumerate(zip(obs.squeeze(0).tolist(), beliefs.tolist())):
	print(f"t={t:02d} obs={o} belief={b}")
